In [1]:
#!/usr/bin/env python3

# core openmm imports
from simtk.openmm import *
from simtk.openmm.app import *
from simtk.unit import *


# mdtraj provides extended trajectory output formats such as HDF5 and NetCDF
import mdtraj as mdt

# ngl view permits visualization of the system
import nglview as ng

Let us run a MD simulation of alanine dipeptide in vacuum using the AMBER 99 forcefield: first we need an input object (e.g. a pdb), and a forcefield definition (several are provided with the OpenMM installation)

In [6]:
# Input Files
print('Parsing PDB file...')
pdb = PDBFile('ala2.pdb')

print('Parsing FF file(s)...')
forcefield = ForceField('amber99sbildn.xml')


Parsing PDB file...
Parsing FF file(s)...


Define simulation parameters here: periodic boundaries if any, nonbonded interaction cutting scheme, rigid constraints for some of the bonds, etc.

In [7]:
# System Configuration
nonbondedMethod = CutoffNonPeriodic
nonbondedCutoff = 1.4*nanometers
constraints = HBonds
constraintTolerance = 0.00001

Create an Integrator : we use 'Langevin' at a temperature of 300 K ; for systems where a simulation box is defined it is also possible to add a barostat and then run in the NPT ensemble

In [8]:
# Integration Options
dt = 0.002*picoseconds
temperature = 300.00*kelvin
friction = 2/picosecond

integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)


An OpenMM platform object is used for providing a level of hardware abstraction : one can choose Reference, CPU, OpenCL or CUDA

In [9]:
platform = Platform.getPlatformByName('CPU')
platformProperties = {'CpuThreads':'1'}

#platform = Platform.getPlatformByName('OpenCL')
#platformProperties = {'OpenCLPrecision':'mixed'}

#platform = Platform.getPlatformByName('CUDA')
#platformProperties = {'CudaPrecision': 'mixed'}


A system object is created from the pdb, the forcefield and some of the simulation parameters above defined : it contains coordinates, velocities,
atom types, and all the data required for energy/force evaluations.

But the computation is performed within a simulation object (using the above defined platform which actually contains the harware dependent compiled code).

In [10]:
# Prepare the Simulation
print('Building system...')
topology = pdb.topology
positions = pdb.positions
system = forcefield.createSystem(topology, nonbondedMethod=nonbondedMethod,
                                 nonbondedCutoff=nonbondedCutoff,
                                 constraints=constraints)

simulation = Simulation(topology, system, integrator,
                        platform, platformProperties)

# set initial positions to what was one obtained from PDB parsing
simulation.context.setPositions(positions)


Building system...


Energy minimization is performed using the L-BFGS method (https://en.wikipedia.org/wiki/Limited-memory_BFGS)

In [ ]:
# Minimize energy
print('Performing energy minimization...')
simulation.minimizeEnergy()

# after minimization of the energy, set initial velocities to random distribution for a given temperature
simulation.context.setVelocitiesToTemperature(temperature)
